In [ ]:
# --- Cell 1 ---
%matplotlib inline
import numpy as np
import random

import jax.numpy as jnp
from jax import jacfwd, jit

import proxsuite

import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="vpython")

In [ ]:
# --- Cell 2 ---
tf = 5.              # total time
dt = 0.05            # simulation time step
S = round(tf/dt) + 1 # number of steps
print(f"S = {S}")

# Parameters
g = 9.81 # gravity (m/s^2)
m1 = 1.  # mass of link 1 (kg)
m2 = 1.  # mass of link 2 (kg)
l1 = 0.5 # length of link 1 (m)
l2 = 0.5 # length of link 2 (m)

# **1. Modeling**

1. Write down a function in Python called `dynamics()` that takes as input the state $x \in \mathbb{R}^{4 \times 1}$ and controls $u \in \mathbb{R}^{2 \times 1}$ and returns $\dot{x} \in \mathbb{R}^{4 \times 1}.$

In [ ]:
# --- Cell 3 ---
def M(q):
    q1, q2 = q[0,0], q[1,0]
    A = np.array([[l1**2 * m1 + l2**2 * m2 + l1**2 * m2 + 2 * l1 * m2 * l2 * np.cos(q2), l2**2 * m2 + l1 * m2 * l2 * np.cos(q2)],
                  [                              l2**2 * m2 + l1 * m2 * l2 * np.cos(q2),                             l2**2 * m2]])

    return A

def C(q, q_dot):
    q1, q2 = q[0,0], q[1,0]
    q1_dot, q2_dot = q_dot[0,0], q_dot[1,0]
    A = np.array([[-2 * q2_dot * l1 * m2 * l2 * np.sin(q2), -q2_dot * l1 * m2 * l2 * np.sin(q2)],
                  [     q1_dot * l1 * m2 * l2 * np.sin(q2),                                   0]])
    return A

def G(q):
    q1, q2 = q[0,0], q[1,0]
    A = np.array([[-g * m1 * l1 * np.sin(q1) - g * m2 * (l1 * np.sin(q1) + l2 * np.sin(q1+q2))],
                  [                       -g * m2 * l2 * np.sin(q1+q2)                        ]])
    return A

In [ ]:
# --- Cell 4 ---
def dynamics(x, u):
    q = np.block([[x[0,0], x[1,0]]]).T
    q_dot = np.block([[x[2,0], x[3,0]]]).T

    lhs = M(q)
    rhs = u - C(q,q_dot) @ q_dot + G(q)
    q_ddot = np.linalg.solve(lhs, rhs)

    return np.block([[q_dot], [q_ddot]])

2. Create a simple visualization of the system (e.g. by using the matplotlib library); you are
free to use any library for the visualization.

In [ ]:
# --- Cell 5 ---
def simulation(xs, x0):
    states = np.array(xs).reshape(-1,4).T
    S = states.shape[1]
    
    fig, ax = plt.subplots(figsize=(8, 8))
    colors = plt.cm.summer(np.linspace(0, 1, 10))

    ax.set_xlim(-1.5, 1.5)
    ax.set_ylim(-1.5, 1.5)
    ax.set_aspect('equal')
    ax.grid(True)
    ax.set_title('Modeling of Double Pendulum System')

    ax.plot([-0.5, 0.5], [0, 0], color='black', lw=1.5)         # balck base-link
    ax.plot([0, 0], [0, -0.5], 'b--', lw=1.5, alpha=0.5)        # link-1 axis
    extension_line, = ax.plot([], [], 'b--', lw=1.5, alpha=0.5) # link-2 axis
    
    rods, = ax.plot([], [], 'o-', lw=1, color=colors[0], markersize=4) # double pendulum
    trace, = ax.plot([], [], '-', lw=1, color='grey', alpha=0.5)       # trajectory trace
    
    x, y = [], []

    def init():
        rods.set_data([], [])
        trace.set_data([], [])
        extension_line.set_data([], [])

        x.clear()
        y.clear()
        
        return rods, trace, extension_line

    def animate(i):
        q1 = states[0, i]
        q2 = states[1, i]

        # Forward Kinematics
        x1 = l1 * np.sin(q1)
        y1 = -l1 * np.cos(q1)
        x2 = x1 + l2 * np.sin(q1 + q2)
        y2 = y1 - l2 * np.cos(q2 + q2)

        rods.set_data([0, x1, x2], [0, y1, y2])
    
        x.append(x2)
        y.append(y2)
        trace.set_data(x[-100:], y[-100:])
        
        x_ext = x1 + 1/2 * np.sin(q1)
        y_ext = y1 - 1.2 * np.cos(q1)
        extension_line.set_data([x1, x_ext], [y1, y_ext])

        return rods, trace, extension_line

    # Create animation
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                   frames=S, interval=50, blit=True)
    anim.save("1-modeling.mp4", writer='ffmpeg', fps=20, dpi=100)
    plt.close()
    
    return HTML(anim.to_jshtml())

3. Discretize the system using **Runge-Kutta 4th Order integration**.

In [ ]:
# --- Cell 6 ---
# Runge-Kutta 4th Order integration
def rk4(x, u, dt):
    f1 = dynamics(x, u)
    f2 = dynamics(x + f1 * dt/2, u)
    f3 = dynamics(x + f2 * dt/2, u)
    f4 = dynamics(x + f3 * dt, u)
    
    return x + dt/6 * (f1 + 2*f2 + 2*f3 + f4)

4. Make sure that the integration works and is correct.

In [ ]:
# --- Cell 7 ---
def pd_controller(x, x_ref, kp=3., kd=2.):
    q = np.block([[x[0,0], x[1,0]]]).T
    q_dot = np.block([[x[2,0], x[3,0]]]).T

    q_ref = np.block([[x_ref[0,0], x_ref[1,0]]]).T
    q_dot_ref = np.block([[x_ref[2,0], x_ref[3,0]]]).T
    
    q_error = q_ref - q
    q_dot_error = q_dot_ref - q_dot
    q_ddot = kp * q_error + kd * q_dot_error

    # Control input
    u = M(q) @ q_ddot + C(q,q_dot) @ q_dot + G(q)
    
    return u

In [ ]:
# --- Cell 8 ---
# Initial state
x0 = np.array([[0., 0., 0., 0.]]).T

# Reference state
x_ref = np.array([[np.pi/7, np.pi/7, 0., 0.]]).T

x = np.copy(x0)
xs = [np.copy(x)]
us = []

for i in range(S):
    # Update control input
    u = pd_controller(x, x_ref)
    us.append(np.copy(u))

    # Update state
    x = rk4(x, u, dt)
    xs.append(np.copy(x))

print("Program found the solution!")
print("Wait for simulation...")
simulation(xs, x0)